In [ ]:
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score
#%pip install wandb
#%pip install git+https://github.com/huggingface/nlp.git@fix-bad-type-in-overflow-check

In [ ]:
QPATH = "Quantlet/4-qode2desc"

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')

sys.path.append('../src')

In [ ]:
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

import nltk
nltk.download('punkt')
import evaluate

import importlib
import preprocessing_utils
importlib.reload(preprocessing_utils)

from sklearn.model_selection import train_test_split

In [ ]:
#with open('../../data/preprocessed/Quantlet/Parsed_Qs_with_code_25062023.pkl', 'rb') as file:
#  df = pickle.load(file)

CLEAN_UP = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model_name = "Salesforce/codet5-base-multi-sum"
sample = 'test'

# tokenization
encoder_max_length = 300
decoder_max_length = 100
RS = 42
LR = 5e-5

EPOCHS = 4
TRAIN_BATCH = 4
EVAL_BATCH  = 4

WARMUP_STEPS  = 500
WEIGHT_DECAY  = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 3

LABEL_SMOOTHING  = 0.1
PREDICT_GENERATE = True


EVAL_COLUMNS = ['eval_loss',
           'eval_rouge1',
           'eval_rouge2',
           'eval_rougeL',
           'eval_rougeLsum',
           'eval_bleu',
           'eval_gen_len']

In [ ]:
model = AutoModelWithLMHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          skip_special_tokens=False)
model.to(device)
print(device)

In [ ]:
def batch_tokenize_preprocess(batch,
                              tokenizer,
                              max_source_length,
                              max_target_length):

    source = batch["input_sequence"]
    target = batch["output_sequence"]

    source_tokenized = tokenizer(
        source,
        padding="max_length",
        truncation=True,
        max_length=max_source_length
    )

    target_tokenized = tokenizer(
        target,
        padding="max_length",
        truncation=True,
        max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}

    # Ignore padding in the loss

    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]

    return batch

In [ ]:
train_dataset = load_dataset("json",
                             data_files="train_dataset_descr.json",
                             field="data",
                             data_dir="../../data/preprocessed/Quantlet/")

if sample!='validation':
  test_dataset = load_dataset("json",
                            data_files="test_dataset_descr.json",
                            field="data",
                            data_dir="../../data/preprocessed/Quantlet/")
else:
  test_dataset = load_dataset("json",
                            data_files="val_dataset_json.json",
                            field="data",
                            data_dir="../../data/preprocessed/Quantlet/")

In [ ]:
train_data_txt = train_dataset['train']
validation_data_txt = test_dataset['train']

In [ ]:
validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batch_size=8,
    batched=True,
    remove_columns=train_data_txt.column_names,
)

In [ ]:
def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [ ]:
def compute_metrics(eval_preds, metrics_list=['rouge', 'bleu']):

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # POST PROCESSING
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    results_dict = {}
    for m in metrics_list:
        metric = evaluate.load(m)

        if m=='bleu':
            result = metric.compute(
              predictions=decoded_preds, references=decoded_labels
           )
        elif m=='rouge':
            result = metric.compute(
                predictions=decoded_preds, references=decoded_labels, use_stemmer=True
            )
        result = {key: value for key, value in result.items() if key!='precisions'}

        prediction_lens = [
            np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
        ]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        results_dict.update(result)
    return results_dict

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=EPOCHS,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=TRAIN_BATCH,
    per_device_eval_batch_size=EVAL_BATCH,
    learning_rate=LR,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    label_smoothing_factor=LABEL_SMOOTHING,
    predict_with_generate=PREDICT_GENERATE,
    logging_dir="logs",
    logging_steps=LOGGING_STEPS,
    save_total_limit=SAVE_TOTAL_LIM,
    report_to=None
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
results_zero_shot = trainer.evaluate()

results_zero_shot_df = pd.DataFrame(data=results_zero_shot, index=[0])[EVAL_COLUMNS]
results_zero_shot_df.loc[0, :] = results_zero_shot_df.loc[0, :].apply(lambda x: round(x, 3))
display(results_zero_shot_df)

In [ ]:
trainer.train()

In [ ]:
results_fine_tune = trainer.evaluate()

results_fine_tune_df = pd.DataFrame(data=results_fine_tune, index=[0])[EVAL_COLUMNS]

results_fine_tune_df.loc[0, :] = results_fine_tune_df.loc[0, :].apply(lambda x: round(x, 3))


display(results_fine_tune_df)

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["input_sequence"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = validation_data_txt.select(range(20))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

In [ ]:
for i, description in enumerate(test_samples["output_sequence"]):
  print('_'*10)
  print(f'Original: {description}')
  print(f'Summary before Tuning: {summaries_before_tuning[i]}')
  print(f'Summary after Tuning: {summaries_after_tuning[i]}')
  print('_'*10)
  print('\n')